In [2]:
model_filepath_to_load = f"../bin/cnn_models/AlexNetLike_CIFAR10_Sparse_Baseline_best_model.h5"
NUM_CLASSES=10

In [15]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
import numpy as np
from sequential import Sequential
from layers.dense import Dense
from layers.conv2d import Conv2D
from layers.flatten import Flatten
from layers.pooling import MaxPooling2D
from layers.dropout import Dropout

In [11]:
print("Loading CIFAR-10 dataset...")
(X_train_full, y_train_full), (X_test, y_test_labels_for_f1) = cifar10.load_data() # y_test is already (10000,1)

print(f"Original X_train_full shape: {X_train_full.shape}")
print(f"Original y_train_full shape: {y_train_full.shape}")
print(f"Original X_test shape: {X_test.shape}")
print(f"Original y_test shape: {y_test_labels_for_f1.shape}")

# Normalize pixel values
X_train_full_norm = X_train_full.astype('float32') / 255.0
X_test_norm = X_test.astype('float32') / 255.0

y_train_full_int = y_train_full.flatten()
y_test_int_for_f1 = y_test_labels_for_f1.flatten()

num_classes = 10

X_train_final, X_val_final, y_train_final_int, y_val_final_int = train_test_split(
    X_train_full_norm, y_train_full_int,
    test_size=0.2,
    random_state=42,
    stratify=y_train_full_int
)

print(f"Final X_train_final shape: {X_train_final.shape}")
print(f"Final y_train_final_int shape: {y_train_final_int.shape}")
print(f"Final X_val_final shape: {X_val_final.shape}")
print(f"Final y_val_final_int shape: {y_val_final_int.shape}")
print(f"Final X_test_norm shape: {X_test_norm.shape}")
print(f"Final y_test_int_for_f1 shape: {y_test_int_for_f1.shape}")

Loading CIFAR-10 dataset...
Original X_train_full shape: (50000, 32, 32, 3)
Original y_train_full shape: (50000, 1)
Original X_test shape: (10000, 32, 32, 3)
Original y_test shape: (10000, 1)
Final X_train_final shape: (40000, 32, 32, 3)
Final y_train_final_int shape: (40000,)
Final X_val_final shape: (10000, 32, 32, 3)
Final y_val_final_int shape: (10000,)
Final X_test_norm shape: (10000, 32, 32, 3)
Final y_test_int_for_f1 shape: (10000,)


In [12]:
custom_cnn_1 = Sequential([
    Conv2D(48, (5, 5), strides=(1,1), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2,2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2,2)),
    Conv2D(192, (3, 3), activation='relu', padding='same'),
    Conv2D(192, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2,2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(),
    Dense(128, activation='relu'),
    Dropout(),
    Dense(NUM_CLASSES, activation='softmax')
])

In [13]:
custom_cnn_1.load_weights(model_filepath_to_load)

HDF5 layer keys: ['conv2d_1', 'conv2d_2', 'conv2d_3', 'conv2d_4', 'conv2d_5', 'dense_1', 'dense_2', 'dense_output', 'dropout_1', 'dropout_2', 'flatten', 'maxpool_1', 'maxpool_2', 'maxpool_3', 'top_level_model_weights']
Loading 2 weight array(s) into layer 0 (type: Conv2D, key: conv2d_1)
Loading 0 weight array(s) into layer 1 (type: MaxPooling2D, key: maxpool_1)
Pooling has no trainable weights — skipping
Loading 2 weight array(s) into layer 2 (type: Conv2D, key: conv2d_2)
Loading 0 weight array(s) into layer 3 (type: MaxPooling2D, key: maxpool_2)
Pooling has no trainable weights — skipping
Loading 2 weight array(s) into layer 4 (type: Conv2D, key: conv2d_3)
Loading 2 weight array(s) into layer 5 (type: Conv2D, key: conv2d_4)
Loading 2 weight array(s) into layer 6 (type: Conv2D, key: conv2d_5)
Loading 0 weight array(s) into layer 7 (type: MaxPooling2D, key: maxpool_3)
Pooling has no trainable weights — skipping
Loading 0 weight array(s) into layer 8 (type: Flatten, key: flatten)
Flatten

In [14]:
def evaluate_macro_f1(y_true, y_pred, dataset_name="Validation"):
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Macro F1-score on {dataset_name} set: {macro_f1:.4f}")
    return macro_f1

In [19]:
from tensorflow.keras import models, layers
keras_model_1 = models.load_model(model_filepath_to_load)
keras_model_1.summary()

Model: "AlexNetLike_CIFAR10_Sparse"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 48)     │         3,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling2D)        │ (None, 16, 16, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling2D)        │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 192)      │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 192)      │       331,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_3 (MaxPooling2D)        │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_output (Dense)            │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,392,460 (5.31 MB)

 Trainable params: 1,392,458 (5.31 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [21]:
y_test_pred_probs = keras_model_1.predict(X_test_norm[:10])
print(y_test_pred_probs)
y_test_pred_classes = np.argmax(y_test_pred_probs, axis=1)

evaluate_macro_f1(y_test_int_for_f1[:10], y_test_pred_classes, "Keras Model Test")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
[[2.13153008e-03 6.14725403e-04 2.53626388e-02 5.07270455e-01
  6.19474472e-03 4.22464490e-01 1.19020063e-02 1.95634030e-02
  1.34554505e-03 3.15042282e-03]
 [1.03306469e-04 3.23111340e-02 1.34843843e-08 4.48175292e-08
  2.89092583e-10 5.09633058e-10 1.94358947e-08 3.17591602e-13
  9.67391074e-01 1.94445878e-04]
 [1.77374884e-01 1.71600711e-02 9.63192247e-03 6.24242937e-03
  3.53841577e-03 1.01711275e-03 1.46964984e-03 4.02606645e-04
  7.60607302e-01 2.25556362e-02]
 [6.24053001e-01 2.36110948e-02 1.28173590e-01 3.69234569e-02
  3.26619148e-02 6.29985146e-03 6.68187579e-03 2.78528850e-03
  1.28191590e-01 1.06183439e-02]
 [2.42260535e-06 4.30908358e-06 1.13619175e-02 6.92239404e-03
  1.43375713e-02 2.03431438e-04 9.67165649e-01 5.71517546e-07
  3.44071339e-07 1.49885966e-06]
 [1.57996581e-03 3.96380527e-03 4.97072525e-02 9.92200077e-02
  3.44464965e-02 1.13144992e-02 7.95871437e-01 4.32350120e-04
  1.11709989e-03 2.34712358e-03]
 [4.99131717e-03 2.

0.7777777777777777

In [ ]:


y_test_pred_probs_custom = custom_cnn_1.predict(X_test_norm[:10])
print(y_test_pred_probs_custom)
y_test_pred_classes_custom = np.argmax(y_test_pred_probs_custom, axis=1)

evaluate_macro_f1(y_test_int_for_f1[:10], y_test_pred_classes_custom, "Custom Model Test")

[[2.13152844e-03 6.14725543e-04 2.53626224e-02 5.07270426e-01
  6.19474297e-03 4.22464602e-01 1.19020074e-02 1.95633781e-02
  1.34554484e-03 3.15042156e-03]
 [1.03306304e-04 3.23112811e-02 1.34843453e-08 4.48174056e-08
  2.89091385e-10 5.09630681e-10 1.94359048e-08 3.17589341e-13
  9.67390888e-01 1.94446197e-04]
 [1.77374932e-01 1.71600755e-02 9.63192979e-03 6.24243629e-03
  3.53842026e-03 1.01711485e-03 1.46965088e-03 4.02607623e-04
  7.60607173e-01 2.25556598e-02]
 [6.24053080e-01 2.36111097e-02 1.28173441e-01 3.69234138e-02
  3.26618888e-02 6.29983984e-03 6.68186811e-03 2.78528541e-03
  1.28191725e-01 1.06183487e-02]
 [2.42260888e-06 4.30909673e-06 1.13619363e-02 6.92240567e-03
  1.43376003e-02 2.03431801e-04 9.67165480e-01 5.71518245e-07
  3.44071926e-07 1.49886212e-06]
 [1.57996684e-03 3.96380678e-03 4.97072374e-02 9.92199864e-02
  3.44465135e-02 1.13145006e-02 7.95871411e-01 4.32350272e-04
  1.11710079e-03 2.34712599e-03]
 [4.99131949e-03 2.37883156e-01 6.68723198e-04 1.17483485e

0.7777777777777777